# Data Normalization

- Normalize MNIST Data 

In [1]:
# !pip install torch torchvision

## 1. Settings
### 1) Import required libraries

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [3]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [4]:
# 정규화는 transform을 통해 가능합니다.
# 여기서 mean, std는 미리 계산된 값입니다.
# 각각이 1개인 이유는 MNIST 데이터의 채널이 하나이기 때문입니다. 

mnist_train = dset.MNIST("./", 
                         train = True,
                         download = True, 
                         transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean = (0.1307, ), std = (0.3081, ))   # 입력 데이터 정규화
                         ]),
                         target_transform = None)
mnist_test = dset.MNIST("./", 
                        train = False,
                        download = True, 
                        transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean = (0.1307, ), std = (0.3081, ))  # 입력 데이터 정규화
                        ]),
                        target_transform = None)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


### 3) Set DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size = batch_size, 
                                           shuffle = True,
                                           num_workers = 2,
                                           drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size = batch_size, 
                                          shuffle = False,
                                          num_workers = 2,
                                          drop_last = True)

## 3. Model & Optimizer

### 1) CNN Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1),    # 28 x 28
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding = 1),   # 28 x 28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                  # 14 x 14
            nn.Conv2d(32, 64, 3, padding = 1),   # 14 x 14
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                   # 7 x 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )        
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

### 2) Loss func & Optimizer

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

cuda:0


## 4. Train 

In [9]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)          

tensor(2.2938, device='cuda:0', grad_fn=<NllLossBackward>)


In [10]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[ 0.2496, -0.2660,  0.3064],
          [-0.2898, -0.1731,  0.1653],
          [ 0.3170,  0.1902,  0.1709]]],


        [[[-0.2637,  0.2160, -0.1221],
          [ 0.1171,  0.2273,  0.0177],
          [ 0.1001,  0.3124,  0.0423]]],


        [[[ 0.0040,  0.2663, -0.0981],
          [ 0.2115,  0.2649, -0.1836],
          [ 0.0386, -0.2988,  0.1430]]],


        [[[-0.0813,  0.1450, -0.1865],
          [ 0.1470,  0.1969, -0.2492],
          [ 0.0353, -0.2010, -0.1598]]],


        [[[-0.1010,  0.2050, -0.0874],
          [-0.1825,  0.0602,  0.2750],
          [ 0.1715, -0.1768,  0.3067]]],


        [[[ 0.1044,  0.1338, -0.1956],
          [ 0.2459, -0.1249,  0.0006],
          [ 0.1920, -0.0393,  0.2026]]],


        [[[ 0.1429,  0.3061,  0.0419],
          [ 0.2579,  0.1707, -0.0712],
          [ 0.1656, -0.2866, -0.2026]]],


        [[[ 0.0849,  0.2138, -0.3232],
          [ 0.1159,  0.0361, -0.1414],
          [-0.2652, -0.2384,  0.0934]]],


        [

## 5. Test

In [11]:
correct = 0
total = 0

with torch.no_grad():
  for image, label in test_loader:
      x = image.to(device)
      y_ = label.to(device)

      output = model.forward(x)
      _, output_index = torch.max(output, 1)

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100 * correct / total))

Accuracy of Test Data: 19.320913314819336
